<a href="https://colab.research.google.com/github/SampMark/ETL-de-dados-da-PNP/blob/main/GitHub_ETL_of_PNP_Data___Pipeline_for_BigQuery___academic_efficiency_dimension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extração, filtragem, tratamento e armazenamento em Big Query dos dados da da Plataforma Nilo Peçanha (PNP) de Instituições Federais de Ensino (IFES) - dimensão eficiência acadêmica**

Este notebook automatiza a extração, tratamento e carga (ETL) dos microdados da Plataforma Nilo Peçanha (PNP) para o Google BigQuery. O fluxo foi projetado em etapas sequenciais para garantir a integridade e a qualidade dos dados.

## **Fluxo de Execução**

1. **Etapa 1: Instalação e Autenticação**: instalação das bibliotecas Python necessárias para a manipulação dos dados e conexão com os serviços Google (`gspread`, `pandas-gbq`, etc.). Autenticação do usuário e montagem do Google Drive para acesso e armazenamento dos arquivos de dados.
2. **Etapa 2: Definição das Funções Principais**: carregamento das funções em memória que realizam as principais tarefas do pipeline: download, descompressão, análise de cabeçalhos, processamento e tratamento dos dados.
3. **Etapa 3: Configuração do Pipeline**: o usuário define os parâmetros da extração através de uma interface interativa:

    * **Período**: define o intervalo de anos (Ano Inicial e Final) para a extração.
    * **Instituição(ões)**: filtra os dados para uma ou mais instituições específicas.
    * **Força atualização**: opção para baixar novamente os arquivos da PNP, ignorando o cache local no Google Drive.

4. **Etapa 4: Download e Análise de Cabeçalhos**: o script baixa os arquivos de dados compactados (`.gz`) da PNP para o Google Drive com base nas configurações da etapa anterior. Em seguida, os arquivos são descompactados e o script realiza uma análise comparativa dos cabeçalhos (colunas) de cada ano, exibindo uma tabela que destaca as diferenças.
    * **Tabela com o comparativo de cabeçalhos dos CSV processados**: permite a analise prévia das colunas a serem extraídas do conjunto de dados (ex: matriculas, servidores).
    * **Ponto de Decisão**: é gerada uma lista com as colunas comuns a todos os arquivos do período, que servirá de sugestão para a próxima etapa.

5. **Etapa 5: Seleção de Colunas e Processamento dos Dados**: esta etapa demanda uma **Ação do Usuário** que deve copiar a lista de colunas sugerida e editá-la conforme a necessidade, em seguida inserir na célula de código desta etapa. O script então processa os arquivos CSV, unificando-os em um único DataFrame (`df_filtrado`), mantendo apenas as colunas selecionadas e aplicando o filtro de instituição.

6. **Etapa 6 a 8: Tratamento, Limpeza e Criação de Métricas (ETL)**: uma sequência de tratamentos é aplicada ao DataFrame (`df_tratado`) unificado:
    * **Padronização de `Nomes de Cursos`**: criação de uma coluna `Cursos` com nomes limpos e padronizados.
    * **Ajustes de colunas**: correção de inconsistências, renomeação de colunas para compatibilidade com o BigQuery e tratamento de valores únicos.
    * **Conversão de Tipos**: ajuste dos tipos de dados (numérico, texto, data) para garantir consistência.
    * **Cálculo de Métricas**: criação de colunas-chave para análise, como `Vagas` e `Inscritos`, aplicando a lógica de desduplicação para evitar somas inflacionadas.
7. **Etapa 9 e 10: Análise e Exportação para o BigQuery**: são realizadas análises estatísticas descritivas e de _outliers_ sobre os dados tratados para verificar a qualidade final. Finalmente, o DataFrame (`df_tratado`) é exportado para uma tabela no Google BigQuery, finalizando o processo de ETL. O script gera um link direto para a tabela criada.

<img src="https://www2.ifal.edu.br/noticias/ifal-se-destaca-na-eficiencia-academica-dos-institutos-federais-do-nordeste/plataforma-nilo-pecanha/@@images/98c1a2a4-6c59-436f-bdce-effa7ae4d539.jpeg" alt="Logo da Plataforma Nilo Peçanha" width="250"/>

In [ ]:
# -*- coding: utf-8 -*-
"""
Extração de microdados da PNP para Big Query

Este notebook automatiza o fluxo de trabalho com os microdados extraídos da Plataforma Nilo Peçanha (PNP),
permitindo a extração de diferentes tabelas e a análise de seus cabeçalhos em cada anos.
"""

# @title **ETAPA 1: Instalação de dependências, importações e autenticação do usuário no Google Drive**

# Instalação de Dependências
!pip install gspread gspread-dataframe oauth2client pandas-gbq --quiet
print("Dependências instaladas com sucesso!")

In [ ]:
# Importação de bibliotecas
import re
import pandas as pd
import numpy as np
import requests
import gspread
import gzip
import shutil
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import auth, drive
from google.auth import default
from pathlib import Path
from typing import List, Dict, Set

In [ ]:
# Autenticação e Montagem do Google Drive
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    drive.mount('/content/drive')
    print("\nAutenticação e montagem do Google Drive realizadas com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro durante a autenticação ou montagem do Drive: {e}")

In [ ]:
# @title **ETAPA 2: Definição das Funções Principais**

def download_pnp_data(table_name: str, start_year: int, end_year: int, force_update: bool):
    """
    Baixa os arquivos da PNP para uma pasta específica no Google Drive.
    O nome do arquivo e a pasta de destino são baseados no table_name.
    """
    drive_folder = Path(f'/content/drive/MyDrive/Coisas do IFRN/Prodes/Indicadores/PNP/{table_name.capitalize()}')
    drive_folder.mkdir(parents=True, exist_ok=True)
    print(f"Verificando arquivos na pasta do Google Drive: {drive_folder}")

    base_url = "https://d236w85zd3t8iw.cloudfront.net/pnp-tests/microdados"

    for year in range(start_year, end_year + 1):
        file_name = f"microdados_{table_name}_{year}.csv.gz"
        url = f"{base_url}/{year}/{file_name}"
        destination = drive_folder / file_name

        if not force_update and destination.exists():
            print(f"✔ O arquivo para {year} ('{file_name}') já existe. Usando o cache do Drive.")
            continue

        try:
            print(f"⬇ Baixando dados para {year} de {url}...")
            with requests.get(url, stream=True) as r:
                r.raise_for_status()
                with open(destination, 'wb') as f:
                    shutil.copyfileobj(r.raw, f)
            print(f"✔ Download de {year} concluído com sucesso.")
        except requests.exceptions.RequestException as e:
            print(f"❌ Falha ao baixar o arquivo para {year}. Erro: {e}. O arquivo pode não existir para este ano.")

def decompress_gz_to_csv(gz_path: Path, out_dir: Path) -> Path:
    """Descompacta cada arquivo .gz para a pasta de trabalho"""
    out_dir.mkdir(parents=True, exist_ok=True)
    # Remove a extensão .gz para obter o nome do arquivo CSV
    csv_out_path = out_dir / gz_path.with_suffix("").name
    print(f"Descompactando: {gz_path.name} -> {csv_out_path.name}")
    with gzip.open(gz_path, "rb") as f_in, open(csv_out_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
    return csv_out_path

def get_header(path: Path, sep: str = ';') -> List[str]:
    """Lê o cabeçalho de cada arquivo CSV descompactado"""
    try:
        return list(pd.read_csv(path, nrows=0, sep=sep, engine='python').columns)
    except Exception as e:
        print(f"Erro ao ler o cabeçalho de {path.name}: {e}")
        return []

def analyze_and_compare_headers(csv_paths: List[Path]) -> pd.DataFrame:
    """
    Cria um DataFrame comparativo de cabeçalhos e sugere colunas comuns.
    """
    if not csv_paths:
        print("Nenhum arquivo CSV para analisar.")
        return pd.DataFrame(), []

    headers_dict = {path.name: get_header(path) for path in csv_paths}

    # Identificar colunas comuns
    sets_of_headers = [set(h) for h in headers_dict.values() if h]
    if not sets_of_headers:
        print("Não foi possível ler nenhum cabeçalho.")
        return pd.DataFrame(), []

    common_columns = sorted(list(sets_of_headers[0].intersection(*sets_of_headers[1:])))

    # Criar DataFrame para comparação visual
    all_columns = sorted(list(set.union(*sets_of_headers)))
    comparison_data = {}
    for col in all_columns:
        comparison_data[col] = [("✔" if col in headers_dict.get(fname, []) else "❌") for fname in headers_dict.keys()]

    comparison_df = pd.DataFrame(comparison_data, index=headers_dict.keys()).transpose()

    print("\n--- Análise de Cabeçalhos Concluída ---")
    print("A tabela abaixo mostra quais colunas estão presentes (✔) ou ausentes (❌) em cada arquivo.")
    display(HTML(comparison_df.to_html()))

    print("\n--- Sugestão de Colunas Comuns ---")
    print(f"Foram encontradas {len(common_columns)} colunas presentes em TODOS os arquivos do período:")
    # Imprime a lista formatada para ser copiada e colada
    print("\nmanter_colunas = [")
    for col in common_columns:
        print(f"    '{col}',")
    print("]")

    return comparison_df, common_columns

def process_to_dataframe(csv_paths: List[Path], columns_to_keep: List[str], institutions: List[str], chunksize: int = 100000, sep: str = ';'):
    """
    Unifica, filtra e concatena os CSVs em um único DataFrame,
    mantendo apenas a lista de colunas fornecida.
    """
    if not csv_paths:
        raise RuntimeError("Nenhum arquivo CSV para processar.")
    if not columns_to_keep:
        raise ValueError("A lista 'columns_to_keep' não pode estar vazia.")

    print(f"\nProcessamento iniciado. Serão importadas {len(columns_to_keep)} colunas pré-definidas.")

    # Encontrar coluna da instituição (considerando inconsistências de codificação)
    col_inst = None
    if 'Instituição' in columns_to_keep:
        col_inst = 'Instituição'
    elif 'InstituiÃ§Ã£o' in columns_to_keep:
        col_inst = 'InstituiÃ§Ã£o'

    if institutions and col_inst:
        print(f"Filtrando pela coluna '{col_inst}' com os valores: {institutions}")
    elif institutions:
        print("AVISO: Filtro de instituição solicitado, mas a coluna 'Instituição' não está na lista de colunas a serem mantidas.")

    institution_map = {'Instituto Federal do Rio Grande do Norte': 'IFRN'}
    df_list = []

    for csv_path in csv_paths:
        print(f"Processando e filtrando: {csv_path.name}")
        try:
            # Lê o cabeçalho do arquivo para saber quais colunas ele realmente tem
            actual_header = get_header(csv_path, sep)
            # Usa apenas as colunas da nossa lista que existem neste arquivo
            cols_to_read = [col for col in columns_to_keep if col in actual_header]

            for chunk in pd.read_csv(csv_path, usecols=cols_to_read, chunksize=chunksize, sep=sep, engine='python', on_bad_lines='warn'):
                if col_inst and col_inst in chunk.columns:
                    chunk[col_inst] = chunk[col_inst].replace(institution_map)
                    if institutions:
                        chunk = chunk[chunk[col_inst].isin(institutions)]

                if not chunk.empty:
                    df_list.append(chunk)
        except Exception as e:
            print(f"  ERRO ao processar o arquivo {csv_path.name}: {e}. Pulando este arquivo.")
            continue

    if not df_list:
        print("AVISO: Nenhum dado encontrado para as instituições selecionadas ou os arquivos estavam vazios.")
        return pd.DataFrame(columns=columns_to_keep)

    final_df = pd.concat(df_list, ignore_index=True)
    # Garante que o DataFrame final tenha todas as colunas da lista, preenchendo com NaN as que não existiam
    final_df = final_df.reindex(columns=columns_to_keep)

    print(f"\nProcesso concluído. DataFrame final criado com {len(final_df):,} linhas e {len(final_df.columns)} colunas.")
    return final_df

In [ ]:
# @title **ETAPA 3: Configuração do Processo e Download (Opção para Código da Unidade)**

# --- Interface Interativa de Configuração ---
style = {'description_width': 'initial'}

# Seleção da tabela
table_name_dropdown = widgets.Dropdown(
    options=['matriculas', 'eficiencia_academica', 'financeiro', 'servidores'],
    value='eficiencia_academica',
    description='Tabela de Dados:',
    style=style
)

# Período de anos
start_year_slider = widgets.IntSlider(value=2017, min=2017, max=2024, step=1, description='Ano Inicial:', style=style)
end_year_slider = widgets.IntSlider(value=2024, min=2017, max=2024, step=1, description='Ano Final:', style=style)

# Opção de forçar atualização
force_update_checkbox = widgets.Checkbox(value=True, description='Forçar atualização (baixar novamente os arquivos existentes)', style=style)

# --- Filtro de instituições por código ---
# O campo pede o código numérico da instituição.
institution_code_text = widgets.Text(
    value='26435', # O valor padrão está preenchido com o código do IFRN = '26435'
    description='Códigos das Instituições (Co Inst):',
    style=style,
    layout=widgets.Layout(width='50%')
)

print("--- Configure os Parâmetros do Pipeline ---")
display(table_name_dropdown)
display(widgets.HBox([start_year_slider, end_year_slider]))
display(force_update_checkbox)
display(institution_code_text)

In [ ]:
# @title **ETAPA 3: Configuração do Processo e Download**

# --- Interface Interativa de Configuração ---
style = {'description_width': 'initial'}

# Seleção da tabela
table_name_dropdown = widgets.Dropdown(
    options=['matriculas', 'eficiencia_academica', 'financeiro', 'servidores'],
    value='eficiencia_academica',
    description='Tabela de Dados:',
    style=style
)

# Período de anos
start_year_slider = widgets.IntSlider(value=2017, min=2017, max=2024, step=1, description='Ano Inicial:', style=style)
end_year_slider = widgets.IntSlider(value=2024, min=2017, max=2024, step=1, description='Ano Final:', style=style)

# Opção de forçar atualização
force_update_checkbox = widgets.Checkbox(value=True, description='Forçar atualização (baixar novamente os arquivos existentes)', style=style)

# --- Filtro de instituições por nome ---
# O campo agora aceita um ou mais nomes de instituições, separados por vírgula.
# O valor padrão já inclui as duas variações para o IFRN.
institution_name_text = widgets.Text(
    value='IFRN, Instituto Federal do Rio Grande do Norte',
    description='Nome da Instituição (use vírgula para múltiplos nomes):',
    style=style,
    layout=widgets.Layout(width='70%') # Largura aumentada para melhor visualização
)

print("--- Configure os Parâmetros do Pipeline ---")
display(table_name_dropdown)
display(widgets.HBox([start_year_slider, end_year_slider]))
display(force_update_checkbox)
display(institution_name_text)

In [ ]:
# @title **ETAPA 4: Download dos arquivos `.gz` e análise dos cabeçalhos dos `.csv` extraídos**

# 1. Pega os valores dos widgets da Etapa 3
table_name = table_name_dropdown.value
start_year = start_year_slider.value
end_year = end_year_slider.value
force_update = force_update_checkbox.value

# Aplica o nome correto do widget e analisa as strings separadas por vírgulas
institutions_str = institution_name_text.value
institutions_list = [inst.strip() for inst in institutions_str.split(',') if inst.strip()]

# 2. Executa o download
download_pnp_data(table_name, start_year, end_year, force_update)

# 3. Prepara os arquivos para a análise
drive_folder = Path(f'/content/drive/MyDrive/Coisas do IFRN/Prodes/Indicadores/PNP/{table_name.capitalize()}')
work_dir = Path.cwd() / "extracted_csvs"
if work_dir.exists(): shutil.rmtree(work_dir)
work_dir.mkdir()

input_files_gz = [drive_folder / f"microdados_{table_name}_{year}.csv.gz" for year in range(start_year, end_year + 1)]
input_files_gz_existing = [f for f in input_files_gz if f.exists()]

all_csvs = []
if input_files_gz_existing:
    for gz_file in input_files_gz_existing:
        all_csvs.append(decompress_gz_to_csv(gz_file, work_dir))
else:
    print("Nenhum arquivo .gz encontrado no Drive para o período e tabela selecionados.")

# 4. Analisa e compara os cabeçalhos
if all_csvs:
    comparison_df, common_columns = analyze_and_compare_headers(all_csvs)
else:
    print("Nenhuma análise de cabeçalho pôde ser feita, pois nenhum arquivo CSV foi descompactado.")

print("\n\n>>> AÇÃO NECESSÁRIA <<<")
print("Copie a lista de colunas comuns sugerida acima (ou edite-a conforme sua necessidade) e cole na célula da 'ETAPA 5' antes de executá-la.")

In [ ]:
# @title **ETAPA 5: Definição das colunas necessárias para extração de `df_filtrado`**

# >>> LISTA DE COLUNAS PARA EXTRAÇÃO (COLE AQUI) <<<
# Exemplo baseado na sugestão da etapa anterior.
# Edite esta lista conforme a necessidade.
# Lista de colunas a serem mantidas no DataFrame final, organizadas por categoria e finalidade.

manter_colunas = [
    # --- IDENTIFICADORES E CÓDIGOS ---
    # Códigos únicos utilizados para referenciar instituições, unidades, matrículas e ciclos.
    'Co Inst', 'Instituição', 'Cod Unidade', 'Unidade de Ensino', 'Código da Unidade de Ensino - SISTEC',
    'Código do Ciclo Matricula', 'Código da Matricula',

    # --- DADOS DE LOCALIZAÇÃO ---
    # Variáveis de localização geográfica.
    'Município', 'Código do Município com DV', 'UF', 'Região',

    # --- DADOS DO CURSO ---
    # Variáveis que descrevem as características do curso ofertado.
    'Modalidade de Ensino', 'Tipo de Curso', 'Tipo de Oferta',  'Eixo Tecnológico', 'Subeixo Tecnológico',
    'Nome de Curso', 'Turno', 'Carga Horaria', 'Carga Horaria Mínima', 'Fonte de Financiamento',
    'Fator Esforço Curso', # Fator de ponderação para cálculo de Matrícula Equivalente

    # --- DADOS DO ALUNO (DEMOGRÁFICOS E SOCIOECONÔMICOS) ---
    # Variáveis com informações pessoais e socioeconômicas do estudante.
    'Sexo', 'Cor / Raça', 'Idade', 'Faixa Etária', 'Renda Familiar',

    # --- DADOS DA MATRÍCULA ---
    # Variáveis que detalham o status e o período do vínculo do aluno com o curso.
    'Ano', 'Matrícula Atendida', 'Situação de Matrícula', 'Categoria da Situação',
    'Data de Inicio do Ciclo', 'Data de Fim Previsto do Ciclo',
    'Data de Ocorrencia da Matricula', 'Mês De Ocorrência da Situação',

    # --- DADOS DE VAGAS E INGRESSO ---
    # Variáveis relacionadas ao processo seletivo, número de vagas e forma de entrada.
    'Total de Inscritos', 'Forma de ingresso', 'Vagas Ofertadas',
    'Vagas Regulares AC', 'Vagas Regulares l1', 'Vagas Regulares l2', 'Vagas Regulares l5', 'Vagas Regulares l6',
    'Vagas Regulares l9', 'Vagas Regulares l10', 'Vagas Regulares l13', 'Vagas Regulares l14',
    'Vagas Extraordinárias AC', 'Vagas Extraordinárias l1', 'Vagas Extraordinárias l2',
    'Vagas Extraordinárias l5', 'Vagas Extraordinárias l6', 'Vagas Extraordinárias l9',
    'Vagas Extraordinárias l10', 'Vagas Extraordinárias l13', 'Vagas Extraordinárias l14'
]

# --- Execução do Processamento ---
df_filtrado = None # Inicializa a variável
if not manter_colunas:
    print("❌ ERRO: A lista 'manter_colunas' está vazia. Preencha-a com as colunas desejadas e execute novamente.")
elif not all_csvs:
    print("❌ ERRO: Nenhum arquivo CSV foi encontrado para processar. Execute a Etapa 4 primeiro.")
else:
    # Cria o DataFrame filtrado com base na seleção de colunas
    df_filtrado = process_to_dataframe(
        csv_paths=all_csvs,
        columns_to_keep=manter_colunas,
        institutions=institutions_list
    )
    display(df_filtrado.head())

# --- Análise do DataFrame Gerado ---
if df_filtrado is not None:
    print("\n--- Análise Detalhada do DataFrame Final ---")
    if df_filtrado.empty:
        print("O DataFrame foi criado, mas está vazio (não contém linhas).")
    else:
        num_rows, num_cols = df_filtrado.shape
        print(f"Dimensões: {num_rows:,} linhas e {num_cols} colunas.")
        print("\nEstrutura e Tipos de Dados:")
        df_filtrado.info()
else:
    print("\nO DataFrame 'df_filtrado' não foi criado. Verifique os erros nas etapas anteriores.")

In [ ]:
# @title **Listagem e análise de valores únicos para as colunas indicadas em `df_filtrado`**
if df_filtrado is not None:
    columns_to_list = ['Ano', 'Unidade de Ensino', 'Eixo Tecnológico', 'Subeixo Tecnológico', 'Matrícula Atendida',
                       'Forma de ingresso', 'Faixa Etária', 'Renda Familiar', 'Fonte de Financiamento',
                       'Tipo de Oferta', 'Tipo de Curso', 'Situação de Matrícula', 'Categoria da Situação',
                       ]

    for col in columns_to_list:
        if col in df_filtrado.columns:
            # Converte a coluna para o tipo 'string' antes de classificar valores exclusivos
            unique_values = sorted(df_filtrado[col].astype(str).unique())
            print(f"Valores únicos em ordem alfabética na coluna '{col}':")
            for value in unique_values:
                print(f"- {value}")
            print("-" * 20) # Separador
        else:
            print(f"A coluna '{col}' não foi encontrada no DataFrame 'df_filtrado'.")
else:
    print("DataFrame 'df_filtrado' não encontrado. Execute as etapas anteriores.")

In [ ]:
# @title **Visualização dos valores únicos da coluna 'Nome de Curso' em `df_filtrado`**

if 'df_filtrado' in locals() and df_filtrado is not None and 'Nome de Curso' in df_filtrado.columns:
    unique_nomes_curso = sorted(df_filtrado['Nome de Curso'].unique())
    print("Valores únicos na coluna 'Nome de Curso' (df_filtrado):")
    print(f"Total de valores únicos: {len(unique_nomes_curso)}")
    for nome in unique_nomes_curso:
        print(f"- {nome}")
elif 'df_filtrado' not in locals() or df_filtrado is None:
    print("DataFrame 'df_filtrado' não encontrado. Execute as etapas anteriores.")
else:
    print("Coluna 'Nome de Curso' não encontrada no DataFrame 'df_filtrado'.")

In [ ]:
# @title **Alicação de prefixos aos nomes de cursos para evitar padronização de nomes iguais, mas de diferentes modalidades/tipos**
def renomear_cursos(row):
    """
    Verifica o 'Tipo de Curso' em uma linha do DataFrame e adiciona o
    prefixo correspondente ao 'Nome de Curso'.
    """
    tipo_curso = row['Tipo de Curso']
    nome_curso = row['Nome de Curso']

    if tipo_curso == 'Licenciatura':
        return f'Licenciatura em {nome_curso}'
    elif tipo_curso == 'Qualificação Profissional (FIC)':
        # Adiciona o prefixo 'FIC em' em vez de 'Qualificação Profissional (FIC) em'
        return f'FIC {nome_curso}'
    elif tipo_curso == 'Tecnologia':
        return f'Tecnologia em {nome_curso}'
    else:
        # Se não for nenhum dos tipos especificados, retorna o nome original
        return nome_curso

# --- Criação da nova coluna 'Cursos' ---
# A função 'renomear_cursos' é aplicada a cada linha (axis=1) do DataFrame.
# O resultado é armazenado na nova coluna 'Cursos'.
df_filtrado['Cursos'] = df_filtrado.apply(renomear_cursos, axis=1)

# --- Exibição do DataFrame final com a nova coluna ---
print("--- DataFrame com a Nova Coluna 'Cursos' ---")
display(df_filtrado)

In [ ]:
# Exibir valores únicos da coluna 'Cursos' em df_filtrado
if 'df_filtrado' in locals() and df_filtrado is not None and 'Cursos' in df_filtrado.columns:
    unique_cursos = sorted(df_filtrado['Cursos'].unique())
    print("Valores únicos na coluna 'Cursos' (df_filtrado):")
    print(f"Total de valores únicos: {len(unique_cursos)}")
    for nome in unique_cursos:
        print(f"- {nome}")
elif 'df_filtrado' not in locals() or df_filtrado is None:
    print("DataFrame 'df_filtrado' não encontrado. Execute as etapas anteriores.")
else:
    print("Coluna 'Cursos' não encontrada no DataFrame 'df_filtrado'.")

In [ ]:
# Calcular e exibir a diferença em valores únicos dos cursos antes e depois do tratamento
if ('df_filtrado' in locals() and df_filtrado is not None and
    'Nome de Curso' in df_filtrado.columns and 'Cursos' in df_filtrado.columns):

    unique_nomes_curso = df_filtrado['Nome de Curso'].unique()
    unique_cursos = df_filtrado['Cursos'].unique()

    num_unique_nomes_curso = len(unique_nomes_curso)
    num_unique_cursos = len(unique_cursos)
    difference = num_unique_nomes_curso - num_unique_cursos

    print(f"Total de valores únicos na coluna 'Nome de Curso' (df_filtrado): {num_unique_nomes_curso}")
    print(f"Total de valores únicos na coluna 'Cursos' (df_filtrado): {num_unique_cursos}")
    print(f"Diferença no número de valores únicos: {difference}")

elif 'df_filtrado' not in locals() or df_filtrado is None:
    print("DataFrame 'df_filtrado' não encontrado. Execute as etapas anteriores.")
else:
    print("As colunas 'Nome de Curso' ou 'Cursos' não foram encontradas no DataFrame 'df_filtrado'.")

In [ ]:
# @title **ETAPA 6: Script para tratamento detalhado dos parâmetros da coluna 'Cursos'**

def padronizar_nome_curso(df, nome_coluna='Cursos'):
    """
    Aplica uma série de regras de padronização a uma coluna de nomes de cursos em um DataFrame do pandas.

    As etapas de limpeza incluem:
    1. Remoção de informações redundantes (prefixos, sufixos de modalidade, etc.).
    2. Normalização de variações complexas e padronização de nomes específicos.
    3. Correção de erros de digitação e variações simples.
    4. Padronização da capitalização para Title Case, preservando conectivos em minúsculo.
    5. Remoção de espaços em branco extras.

    Args:
        df (pd.DataFrame): O DataFrame contendo os dados.
        nome_coluna (str): O nome da coluna com os nomes dos cursos a serem padronizados.

    Returns:
        pd.DataFrame: O DataFrame com a coluna de nomes de cursos padronizada.
    """
    # Copia a coluna para evitar alterar o DataFrame original durante o processo
    cursos_padronizados = df[nome_coluna].copy().astype(str)

    # --- Etapa 1: Remover informações redundantes (prefixos e sufixos) ---
    prefixos_para_remover = [
        r'^fic\s*-\s*',
        r'^pós graduação em\s*',
        r'^pós-graduação em\s*',
        r'^especialização \(lato sensu\)\s*-\s*',
        r'^especialização\s*-\s*',
        r'^mestrado\s*-\s*',
        r'^doutorado\s*-\s*',
        r'^mestrado profissional\s*-\s*',
        r'^fic qualificação profissional\s*-\s*',
        r'curso de Pós-graduação Lato Sensu '
    ]
    for prefixo in prefixos_para_remover:
        cursos_padronizados = cursos_padronizados.str.replace(prefixo, '', regex=True, flags=re.IGNORECASE)

    # Remove sufixos de modalidade
    sufixos_para_remover = [
        r',\s*na modalidade ead$',
        r',\s*na modalidade semipresencial$'
    ]
    for sufixo in sufixos_para_remover:
        cursos_padronizados = cursos_padronizados.str.replace(sufixo, '', regex=True, flags=re.IGNORECASE)

    # --- Etapa 2: Normalização de variações complexas ---
    # Este dicionário usa regex para encontrar padrões e substituir pelo nome padronizado.
    # A ordem é importante: regras mais específicas devem vir antes.
    regras_de_normalizacao = {
        # --- Doutorados ---
        # --- AJUSTE ---: Adicionada regra para Doutorado Acadêmico em Educação Profissional.
        r'.*Doutorado Acad[êe]mico em Educaç[ãa]o Profissional.*': 'Doutorado em Educação Profissional e Tecnológica',
        # --- AJUSTE ---: Adicionada regra para Doutorado em Desenvolvimento Educacional e Social.
        # r'.*Doutorado.*Desenvolvimento Educacional e Social.*': 'Doutorado em Desenvolvimento Educacional e Social',
        r'.*Doutorado em Ensino.*(Rede Nordeste|renoen).*': 'Doutorado em Ensino (RENOEN)',
        r'Rede Nordeste de Ensino - RENOEN': 'Doutorado em Ensino (RENOEN)',

        # --- Mestrados ---
        r'.*Mestrado.*Educação Profissional e Tecnológica.*': 'Mestrado em Educação Profissional e Tecnológica (ProfEPT)',
        r'Mestrado Acadêmico em Educação Profissional': 'Mestrado em Educação Profissional e Tecnológica (ProfEPT)',
        r'Mestrado em Ensino': 'Mestrado em Ensino (Posensino)',
        r'Mestrado Profissional - Recursos Naturais': 'Mestrado Profissional em Uso Sustentável dos Recursos Naturais',
        r'Uso Sustentável dos Recursos Naturais': 'Mestrado Profissional em Uso Sustentável dos Recursos Naturais',

        # --- Especializações ---
        r'Especialização em Educação Ambiental e Geografia do Semi-Árido': 'Especialização em Educação Ambiental e Geografia do Semiárido',
        r'Educação Ambiental e Geografia do Semiarido': 'Especialização em Educação Ambiental e Geografia do Semiárido',
        r'.*Ci[êe]ncias Humanas e Saberes Contempor[âa]neos.*': 'Especialização em Ciências Humanas e Saberes Contemporâneos para a Educação',
        r'Ciências Humanas e Competências Contemporâneas para a Educação': 'Especialização em Ciências Humanas e Competências Contemporâneas para a Educação',
        r'Docência para a Educação Profissional e Tecnológica (DocentEPT)': 'Especialização em Docência na Educação Profissional e Tecnológica (DocentEPT)',
        r'Especialização em Eja No Contexto da Diversidade': 'Especialização em Educação de Jovens e Adultos no Contexto da Diversidade',
        r'Engenharia de Segurança do Trabalho': 'Especialização em Engenharia de Segurança do Trabalho',
        r'Ensino de Ciências Biológicas': 'Especialização em Ensino de Ciências Biológicas',
        r'Ensino de Ciências Naturais e Matemática': 'Especialização em Ensino de Ciências Naturais e Matemática',
        r'Ensino de Geociências': 'Especialização em Ensino de Geociências',
        r'Ensino de Teatro': 'Especialização em Ensino de Teatro',
        r'Especialização em Ciência e Tecnologia de Alimentos, Na Modalidade Ead': 'Especialização em Ciência e Tecnologia de Alimentos',
        r'Especialização em Ensino de Língua Portuguesa e Matemática Transdisciplinar': 'Especialização em Ensino da Língua Portuguesa e Matemática numa Abordagem Transdisciplinar',
        r'Especialização em Docência para a Educação Profissional e Tecnológica – Docentept - Uab': 'Especialização em Docência na Educação Profissional e Tecnológica (DocentEPT)',
        r'Estudos Linguísticos e Literários': 'Especialização em Estudos Linguísticos e Literários',
        r'Especialização em Práticas Assertivas da Educação Profissional Integrada À Educação de Jovens e Adultos - Com Ênfase em Didática': 'Especialização em Práticas Assertivas em Didática e Gestão da Educação Profissional integrada à EJA',
        r'^Gestão Ambiental$': 'Especialização em Gestão Ambiental',
        r'Licenciatura em Licenciatura para a Educação Profissional, Científica e Tecnológica': 'Licenciatura em Formação Pedagógica para a Educação Básica, Profissional e Tecnológica'
     }

    for padrao, nome_correto in regras_de_normalizacao.items():
        cursos_padronizados = cursos_padronizados.str.replace(padrao, nome_correto, regex=True, flags=re.IGNORECASE)

    # --- Etapa 3: Corrigir erros de digitação e variações simples ---
    correcoes_simples = {
        'Conteporaneidade': 'Contemporaneidade',
        'À': 'à', # Corrige capitalização de crase
        'Semi-Árido': 'Semiárido',
        'Eja No Contexto da Diversidade': 'EJA no Contexto da Diversidade',
        'para O Ensino Médio': 'para o Ensino Médio',
        'desportiva e de Lazer': 'Desportiva e de Lazer'
    }
    for erro, correcao in correcoes_simples.items():
        cursos_padronizados = cursos_padronizados.str.replace(erro, correcao, regex=False)

    # --- Etapa 4: Padronização da capitalização ---
    # Converte para Title Case
    cursos_padronizados = cursos_padronizados.str.title()

    # Lista de conectivos e palavras curtas que devem ser minúsculas
    conectivos_minusculos = ['de', 'da', 'do', 'dos', 'das', 'em', 'e', 'para', 'o', 'a', 'à', 'os', 'as', 'no', 'na', 'nos', 'nas', 'com']

    def apply_title_case_with_exceptions(name):
        # Garante que siglas como IFRN, EJA, EAD, etc., fiquem em maiúsculas
        name = re.sub(r'\b(If(rn)?|Eja|Ead|Fic|Ppi|Pcd|Renoen|Profept|Docentept|Posensino)\b', lambda m: m.group().upper(), name, flags=re.IGNORECASE)
        words = name.split()
        processed_words = [word.lower() if word.lower() in conectivos_minusculos and i > 0 else word for i, word in enumerate(words)]
        return ' '.join(processed_words)

    cursos_padronizados = cursos_padronizados.apply(apply_title_case_with_exceptions)

    # --- Etapa 5: Remover espaços em branco extras ---
    cursos_padronizados = cursos_padronizados.str.strip()
    cursos_padronizados = cursos_padronizados.str.replace(r'\s+', ' ', regex=True)

    # Atribui a coluna padronizada de volta ao DataFrame
    df_tratado = df.copy()
    df_tratado[nome_coluna] = cursos_padronizados

    return df_tratado

# Aplica a função de padronização à coluna 'Cursos'
df_tratado = padronizar_nome_curso(df_filtrado.copy(), 'Cursos')

if df_tratado is not None:
    num_rows, num_cols = df_tratado.shape
    print(f"DataFrame 'df_tratado' criado como uma cópia de 'df_filtrado' tem {num_rows:,} linhas e {num_cols} colunas.")
else:
    print("DataFrame 'df_tratado' não encontrado. Execute as etapas anteriores.")

# Exibir valores únicos em ordem alfabética na coluna 'Cursos'
if df_tratado is not None and 'Cursos' in df_tratado.columns:
    unique_cursos = sorted(df_tratado['Cursos'].unique())
    print("Valores únicos na coluna 'Cursos':")
    print(f"Total de valores únicos na coluna 'Cursos' após limpeza: {len(unique_cursos)}")
    for curso in unique_cursos:
        print(f"- {curso}")
elif df_tratado is None:
    print("DataFrame 'df_tratado' não encontrado. Execute as etapas anteriores.")
else:
    print("Coluna 'Cursos' não encontrada no DataFrame.")


In [ ]:
# @title **Calcular e exibir a diferença em valores únicos antes e após tratamento dos nomes de cursos**
if ('df_filtrado' in locals() and df_filtrado is not None and 'Nome de Curso' in df_filtrado.columns and
    'df_tratado' in locals() and df_tratado is not None and 'Cursos' in df_tratado.columns):

    unique_nomes_curso_filtrado = df_filtrado['Nome de Curso'].unique()
    unique_cursos_tratado = df_tratado['Cursos'].unique()

    num_unique_nomes_curso_filtrado = len(unique_nomes_curso_filtrado)
    num_unique_cursos_tratado = len(unique_cursos_tratado)
    difference = num_unique_nomes_curso_filtrado - num_unique_cursos_tratado

    print(f"Total de valores únicos na coluna 'Nome de Curso' (df_filtrado): {num_unique_nomes_curso_filtrado}")
    print(f"Total de valores únicos na coluna 'Cursos' (df_tratado): {num_unique_cursos_tratado}")
    print(f"Diferença no número de valores únicos: {difference}")

elif 'df_filtrado' not in locals() or df_filtrado is None or 'Nome de Curso' not in df_filtrado.columns:
    print("DataFrame 'df_filtrado' ou a coluna 'Nome de Curso' não encontrada. Execute as etapas anteriores.")
elif 'df_tratado' not in locals() or df_tratado is None or 'Cursos' not in df_tratado.columns:
     print("DataFrame 'df_tratado' ou a coluna 'Cursos' não encontrada. Execute as etapas anteriores.")

In [ ]:
# @title **Mapear os valores em 'Matrícula Atendida' e converter para `booleano`**
if 'df_tratado' in locals() and df_tratado is not None and 'Matrícula Atendida' in df_tratado.columns:
    print("Convertendo a coluna 'Matrícula Atendida' para booleano...")
    mapping = {"Sim": 1, "Y": 1, "Não Informado": 0}
    # Certifica-se de que a coluna é do tipo objeto para lidar com tipos mistos antes do mapeamento
    df_tratado['Matrícula Atendida'] = df_tratado['Matrícula Atendida'].astype(str).map(mapping).astype(bool)
    print("Conversão concluída.")
    print("\nContagem dos valores na coluna 'Matrícula Atendida' após conversão:")
    display(df_tratado['Matrícula Atendida'].value_counts())
elif 'df_tratado' not in locals() or df_tratado is None:
    print("DataFrame 'df_tratado' não encontrado. Execute as etapas anteriores.")
else:
    print("Coluna 'Matrícula Atendida' não encontrada no DataFrame 'df_tratado'.")

In [ ]:
# @title **Ajuste/correção de parâmetros em diversas colunas, renomeia adequadamente as colunas em `df_tratado` a ser exportado para o BigQuery**

# Renomeia colunas adequadamente para o BigQuery para evitar 'KeyErrors'
df_tratado = df_tratado.rename(columns={
    'Carga Horaria': 'Carga_Horaria',
    'Carga Horaria Mínima': 'Carga_Horaria_Minima',
    'Categoria da Situação': 'Categoria_da_Situacao',
    'Co Inst': 'Co_Inst',
    'Cod Unidade': 'Cod_Unidade',
    'Cor / Raça': 'Cor_Raça',
    'Código da Matricula': 'Codigo_da_Matricula',
    'Código da Unidade de Ensino - SISTEC': 'Codigo_da_Unidade_de_Ensino_-_SISTEC',
    'Código do Ciclo Matricula': 'Codigo_do_Ciclo_Matricula',
    'Código do Município com DV': 'Codigo_do_Municipio_com_DV',
    'Data de Fim Previsto do Ciclo': 'Data_de_Fim_Previsto_do_Ciclo',
    'Data de Inicio do Ciclo': 'Data_de_Inicio_do_Ciclo',
    'Data de Ocorrencia da Matricula': 'Data_de_Ocorrencia_da_Matricula',
    'Situação de Matrícula': 'Situacao_de_Matricula',
    'Subeixo Tecnológico': 'Subeixo_Tecnologico',
    'Matrícula Atendida': 'Matricula_Atendida',
    'Eixo Tecnológico': 'Eixo_Tecnologico',
    'Faixa Etária': 'Faixa_Etaria',
    'Fator Esforço Curso': 'Fator_Esforco_Curso',
    'Fonte de Financiamento': 'Fonte_de_Financiamento',
    'Forma de ingresso': 'Forma_de_Ingresso',
    'Instituição': 'Instituicao',
    'Mês De Ocorrência da Situação': 'Mes_De_Ocorrencia_da_Situacao',
    'Município': 'Municipio',
    'Modalidade de Ensino': 'Modalidade_Educacional',
    'Tipo de Curso': 'Nivel_Educacional',
    'Tipo de Oferta': 'Modalidade_de_Curso',
    'Nome de Curso': 'Nome_de_Curso',
    'Total de Inscritos': 'Total_de_Inscritos',
    'Região': 'Regiao',
    'Renda Familiar': 'Renda_Familiar',
    'Sexo': 'Sexo',
    'Unidade de Ensino': 'Campus_do_IFRN',
    'Vagas Extraordinárias AC': 'Vagas_Extraordinarias_AC',
    'Vagas Extraordinárias l1': 'Vagas_Extraordinarias_l1',
    'Vagas Extraordinárias l10': 'Vagas_Extraordinarias_l10',
    'Vagas Extraordinárias l13': 'Vagas_Extraordinarias_l13',
    'Vagas Extraordinárias l14': 'Vagas_Extraordinarias_l14',
    'Vagas Extraordinárias l2': 'Vagas_Extraordinarias_l2',
    'Vagas Extraordinárias l5': 'Vagas_Extraordinarias_l5',
    'Vagas Extraordinárias l6': 'Vagas_Extraordinarias_l6',
    'Vagas Extraordinárias l9': 'Vagas_Extraordinarias_l9',
    'Vagas Ofertadas': 'Vagas_Ofertadas',
    'Vagas Regulares AC': 'Vagas_Regulares_AC',
    'Vagas Regulares l1': 'Vagas_Regulares_l1',
    'Vagas Regulares l10': 'Vagas_Regulares_l10',
    'Vagas Regulares l13': 'Vagas_Regulares_l13',
    'Vagas Regulares l14': 'Vagas_Regulares_l14',
    'Vagas Regulares l2': 'Vagas_Regulares_l2',
    'Vagas Regulares l5': 'Vagas_Regulares_l5',
    'Vagas Regulares l6': 'Vagas_Regulares_l6',
    'Vagas Regulares l9': 'Vagas_Regulares_l9',
})


# Correção de parâmetros na coluna 'Eixo Tecnológico'
replacements_eixo = {
    'GESTÃO E NEGÓCIOS': 'Gestão e Negócios',
    'DESENVOLVIMENTO EDUCACIONAL E SOCIAL': 'Desenvolvimento Educacional e Social'
}
if 'Eixo_Tecnologico' in df_tratado.columns:
  df_tratado['Eixo_Tecnologico'] = df_tratado['Eixo_Tecnologico'].replace(replacements_eixo)

# Correção de parâmetros na coluna 'Subeixo Tecnológico'
replacements_subeixo = {
    'AMBIENTE E SAÚDE': 'Ambiente e Saúde'
}
if 'Subeixo_Tecnologico' in df_tratado.columns:
  df_tratado['Subeixo_Tecnologico'] = df_tratado['Subeixo_Tecnologico'].replace(replacements_subeixo) # Changed from replacements_eixo

# Correção de parâmetros em 'Campus_do_IFRN'
replacements_unidade = {
    'Campus Ceará-mirim': 'Ceará-Mirim',
    'Campus Pau Dos Ferros': 'Pau dos Ferros'
}
if 'Campus_do_IFRN' in df_tratado.columns:
    # Aplicação de regras
    df_tratado['Campus_do_IFRN'] = df_tratado['Campus_do_IFRN'].replace(replacements_unidade)
    # Remoção d palavras 'Campus ' e 'Campus Avançado ' de 'Campus_do_IFRN'
    df_tratado['Campus_do_IFRN'] = df_tratado['Campus_do_IFRN'].str.replace('Campus Avançado ', '', regex=False)
    df_tratado['Campus_do_IFRN'] = df_tratado['Campus_do_IFRN'].str.replace('Campus ', '', regex=False)

# Renomeia parâmetros na coluna 'Forma de ingresso'
replacements_ingresso = {
    'AC': 'AC: Ampla Concorrência',
    'l1': 'L1: Renda Baixa',
    'l2': 'L2: Renda Baixa + PPI',
    'l5': 'L5: Ensino Médio em Escolas Pública',
    'l6': 'L6: PPI (Pretos, Pardos e Indígenas)',
    'l9': 'L9: Renda Baixa + PcD',
    'l10': 'L10: Renda Baixa + PPI + PcD',
    'l13': 'L13: PcD (Independente de Renda)',
    'l14': 'L14: PcD + PPI (Independente de Renda)',
    'LB_EP': 'LB EP: Baixa Renda + Ensino Médio em Escola Pública',
    'LB_PCD': 'LB PCD: Baixa Renda + PcD',
    'LB_PPI': 'LB PPI: Baixa Renda + PPI',
    'LB_Q': 'LB Q: Baixa Renda + Quilombolas + Ensino Médio em Escola Pública',
    'LI_EP': 'LI EP: Ensino Médio em Escola Pública (Independentemente da Renda)',
    'LI_PCD': 'LI PCD: PcD (Independente de Renda)',
    'LI_PPI': 'LI PPI: PPI (Independente de Renda)',
    'LI_Q': 'LI Q: Quilombolas + Ensino Médio em Escola Pública (Independente da Renda)',
    'nan': 'NI: Não informado'
}

if 'Forma_de_Ingresso' in df_tratado.columns:
    # Certifica que a coluna seja do tipo objeto para lidar com substituições de strings, incluindo 'nan'
    df_tratado['Forma_de_Ingresso'] = df_tratado['Forma_de_Ingresso'].astype(str).replace(replacements_ingresso)

# Adiciona padronização em parâmetro da coluna 'Categoria da Situação'
replacements_categoria_situacao = {
    'Em curso': 'Em Curso',
}
if 'Categoria_da_Situacao' in df_tratado.columns:
    df_tratado['Categoria_da_Situacao'] = df_tratado['Categoria_da_Situacao'].replace(replacements_categoria_situacao)

# Substituir em 'Renda_Familiar' as ocorrências de 'S/I' para 'Não declarada'
df_tratado['Renda_Familiar'] = df_tratado['Renda_Familiar'].replace('S/I', 'Não declarada')

# Renomear categorias em 'Renda_Familiar'
renomear_renda = {
    '0<RFP<=0,5': '(1) 0<RFP<=0,5',
    '0,5<RFP<=1':  '(2) 0,5<RFP<=1',
    '1<RFP<=1,5': '(3) 1<RFP<=1,5',
    '1,5<RFP<=2,5': '(4) 1,5<RFP<=2,5',
    '2,5<RFP<=3,5': '(5) 2,5<RFP<=3,5',
    'RFP>3,5': '(6) RFP>3,5',
    'Não declarada': '(7) Não declarada'
}

# Aplicar a renomeação utilizando o método .replace() à coluna
df_tratado['Renda_Familiar'] = df_tratado['Renda_Familiar'].replace(renomear_renda)


# Reposiona 'Cursos' depois de 'Nome_de_Curso'
if 'Cursos' in df_tratado.columns and 'Nome_de_Curso' in df_tratado.columns:
    cols = df_tratado.columns.tolist()
    # Find the index of 'Cursos' and 'Nome_de_Curso'
    cursos_index = cols.index('Cursos')
    nome_curso_index = cols.index('Nome_de_Curso')

    # Remove 'Cursos' from its current position
    cursos_col = cols.pop(cursos_index)

    # Insert 'Cursos' after 'Nome_de_Curso'
    cols.insert(nome_curso_index + 1, cursos_col)

    # Reindex the DataFrame with the new column order
    df_tratado = df_tratado[cols]
    print("Coluna 'Cursos' reposicionada após 'Nome_de_Curso'.")
elif 'Cursos' not in df_tratado.columns:
    print("Aviso: Coluna 'Cursos' não encontrada para reposicionamento.")
elif 'Nome_de_Curso' not in df_tratado.columns:
    print("Aviso: Coluna 'Nome_de_Curso' não encontrada para servir de referência.")


# Exibir informações sobre as colunas e tipos de dados de 'df_tratado'
if df_tratado is not None:
    print("Informações sobre as colunas e tipos de dados do DataFrame 'df_tratado':")
    df_tratado.info()
else:
    print("DataFrame 'df_tratado' não encontrado. Execute as etapas anteriores.")

In [ ]:
# @title **Verificação de valores únicos para as colunas indicadas**
if df_tratado is not None:
    columns_to_list = ['Campus_do_IFRN', 'Eixo_Tecnologico', 'Forma_de_Ingresso', 'Renda_Familiar', 'Categoria_da_Situacao', 'Situacao_de_Matricula']

    for col in columns_to_list:
        if col in df_tratado.columns:
            # Converte a coluna para o tipo 'string' antes de classificar valores exclusivos
            unique_values = sorted(df_tratado[col].astype(str).unique())
            print(f"Valores únicos em ordem alfabética na coluna '{col}':")
            for value in unique_values:
                print(f"- {value}")
            print("-" * 20) # Separador
        else:
            print(f"A coluna '{col}' não foi encontrada no DataFrame 'df_tratado'.")
else:
    print("DataFrame 'df_tratado' não encontrado. Execute as etapas anteriores.")

In [ ]:
# @title **Tratamento dos tipos de dados das colunas de `df_tratado`**
def converter_tipos_de_dados(df: pd.DataFrame, colunas_para_int: list, colunas_para_str: list, colunas_para_data: dict, colunas_para_float: dict) -> pd.DataFrame:
    """
    Converte os tipos de dados de colunas especificadas em um DataFrame.

    Args:
        df (pd.DataFrame): O DataFrame a ser modificado.
        colunas_para_int (list): Uma lista de nomes de colunas para converter para o tipo Int64 (inteiro que suporta nulos).
        colunas_para_str (list): Uma lista de nomes de colunas para converter para o tipo string.
        colunas_para_data (dict): Um dicionário onde as chaves são os nomes das colunas
                                  para converter para data e os valores são os formatos esperados (ex: {'coluna': '%d/%m/%Y'}).
        colunas_para_float (dict): Um dicionário onde as chaves são os nomes das colunas
                                   para converter para float e os valores são os caracteres a serem substituídos
                                   antes da conversão (ex: {'coluna': ','}).

    Returns:
        pd.DataFrame: Uma cópia do DataFrame com os tipos de dados convertidos.
    """
    if not isinstance(df, pd.DataFrame):
        print("Erro: O objeto fornecido não é um DataFrame do pandas.")
        return None

    # Cria uma cópia para evitar modificar o DataFrame original
    df_processado = df.copy()

    print("--- Iniciando conversão de tipos de dados ---")

    # --- Conversão para Inteiro (Int64) ---
    print("\nConvertendo colunas para Inteiro:")
    for col in colunas_para_int:
        if col in df_processado.columns:
            try:
                # Converte para float primeiro para lidar com NaN, depois para Int64 (que suporta NaN)
                df_processado[col] = pd.to_numeric(df_processado[col], errors='coerce').astype('Int64')
                print(f"  ✔ Coluna '{col}' convertida para Int64.")
            except Exception as e:
                print(f"  ❌ Erro ao converter a coluna '{col}': {e}")
        else:
            print(f"  - AVISO: Coluna '{col}' não encontrada no DataFrame.")

    # --- Conversão para String ---
    print("\nConvertendo colunas para String:")
    for col in colunas_para_str:
        if col in df_processado.columns:
            try:
                # Converte valores nulos para uma string vazia antes de mudar o tipo
                df_processado[col] = df_processado[col].fillna('').astype(str)
                print(f"  ✔ Coluna '{col}' convertida para string.")
            except Exception as e:
                print(f"  ❌ Erro ao converter a coluna '{col}': {e}")
        else:
            print(f"  - AVISO: Coluna '{col}' não encontrada no DataFrame.")

    # --- Conversão para Data ---
    print("\nConvertendo colunas para Data:")
    for col, fmt in colunas_para_data.items():
        if col in df_processado.columns:
            try:
                df_processado[col] = pd.to_datetime(df_processado[col], format=fmt, errors='coerce')
                print(f"  ✔ Coluna '{col}' convertida para data com formato '{fmt}'.")
            except Exception as e:
                print(f"  ❌ Erro ao converter a coluna '{col}' para data: {e}")
        else:
            print(f"  - AVISO: Coluna '{col}' não encontrada no DataFrame.")

    # --- Conversão para Float (float64) ---
    print("\nConvertendo colunas para Float:")
    for col, char_to_replace in colunas_para_float.items():
        if col in df_processado.columns:
            try:
                # Substitui o caractere e converte para numérico
                df_processado[col] = df_processado[col].astype(str).str.replace(char_to_replace, '.', regex=False)
                df_processado[col] = pd.to_numeric(df_processado[col], errors='coerce').astype('float64')
                print(f"  ✔ Coluna '{col}' convertida para float64.")
            except Exception as e:
                print(f"  ❌ Erro ao converter a coluna '{col}' para float64: {e}")
        else:
            print(f"  - AVISO: Coluna '{col}' não encontrada no DataFrame.")

    print("\n--- Conversão concluída ---")
    return df_processado

# Define as listas de colunas
colunas_int = [
    'Carga_Horaria_Minima', 'Idade', 'Total_de_Inscritos',
    'Vagas_Extraordinarias_AC', 'Vagas_Extraordinarias_l1', 'Vagas_Extraordinarias_l10',
    'Vagas_Extraordinarias_l13', 'Vagas_Extraordinarias_l14', 'Vagas_Extraordinarias_l2',
    'Vagas_Extraordinarias_l5', 'Vagas_Extraordinarias_l6', 'Vagas_Extraordinarias_l9',
    'Vagas_Ofertadas', 'Vagas_Regulares_AC', 'Vagas_Regulares_l1', 'Vagas_Regulares_l10',
    'Vagas_Regulares_l13', 'Vagas_Regulares_l14', 'Vagas_Regulares_l2', 'Vagas_Regulares_l5',
    'Vagas_Regulares_l6', 'Vagas_Regulares_l9'
]
colunas_str = [
    'Cod_Unidade',
    'Co_Inst',
    'Codigo_da_Matricula',
    'Codigo_da_Unidade_de_Ensino_-_SISTEC',
    'Codigo_do_Ciclo_Matricula',
    'Codigo_do_Municipio_com_DV',
    'Eixo_Tecnologico',
    'Forma_de_Ingresso',
    'Instituicao',
    'Modalidade_Educacional',
    'Nivel_Educacional',
    'Modalidade_de_Curso',
    'Matricula_Atendida',
    'Renda_Familiar',
    'Subeixo_Tecnologico',
    'Categoria_da_Situacao',
]
colunas_data = {
    'Data_de_Fim_Previsto_do_Ciclo': '%d/%m/%Y',
    'Data_de_Inicio_do_Ciclo': '%d/%m/%Y',
    'Data_de_Ocorrencia_da_Matricula': '%d/%m/%Y'
}

colunas_float = {
    'Fator_Esforco_Curso': ',' # Substituir vírgula por ponto
}

# Chama a função e armazena o resultado em um novo DataFrame
df_tratado = converter_tipos_de_dados(df_tratado, colunas_int, colunas_str, colunas_data, colunas_float)

# Verifica os tipos de dados após a conversão (opcional)
if df_tratado is not None:
  print("\nTipos de dados após a conversão:")
  colunas_convertidas = colunas_int + colunas_str + list(colunas_data.keys()) + list(colunas_float.keys())
  print(df_tratado[colunas_convertidas].dtypes)

In [ ]:
# @title **Exibir informações sobre as colunas e tipos de dados de `df_tratado`**
if df_tratado is not None:
    print("Informações sobre as colunas e tipos de dados do DataFrame 'df_tratado':")
    df_tratado.info()
else:
    print("DataFrame 'df_tratado' não encontrado. Execute as etapas anteriores.")

In [ ]:
# @title **ETAPA 8: Cálculo de métricas específicas a partir dos microdados da PNP**

# Código para calcular os totais de 'Vagas' e 'Inscritos' a partir dos microdados da PNP,
# preparando-os para agregação em ferramentas de BI como Looker Studio ou Power BI.

# Reconstrução dos dados a partir das colunas que detalham a distribuição de vagas por cotas.
COL_VAGAS_REG = [
    "Vagas_Regulares_AC","Vagas_Regulares_l1","Vagas_Regulares_l2",
    "Vagas_Regulares_l5","Vagas_Regulares_l6","Vagas_Regulares_l9",
    "Vagas_Regulares_l10","Vagas_Regulares_l13","Vagas_Regulares_l14",
]
COL_VAGAS_EXT = [
    "Vagas_Extraordinarias_AC","Vagas_Extraordinarias_l1","Vagas_Extraordinarias_l2",
    "Vagas_Extraordinarias_l5","Vagas_Extraordinarias_l6","Vagas_Extraordinarias_l9",
    "Vagas_Extraordinarias_l10","Vagas_Extraordinarias_l13","Vagas_Extraordinarias_l14",
]
COL_VAGAS_TODAS = COL_VAGAS_REG + COL_VAGAS_EXT

# --- PASSO 1: Reconstrução do Total de Vagas (Fallback para dados ausentes) ---
# Esta etapa resolve um problema comum de qualidade nos dados da PNP, em que a coluna
# principal 'Vagas_Ofertadas' pode estar nula ou zerada, enquanto os detalhes por cota estão preenchidos.
# O código soma os detalhes para criar um total confiável.

# Certifica se as colunas relevantes são numéricas antes de somar (opcional)
for col in COL_VAGAS_TODAS:
    if col in df_tratado.columns:
        df_tratado[col] = pd.to_numeric(df_tratado[col], errors='coerce').astype('Int64')

df_tratado["Vagas_categorias_soma"] = df_tratado[COL_VAGAS_TODAS].fillna(0).sum(axis=1).astype('Int64')

# --- PASSO 2: Criação de uma Coluna de Vagas Confiável ---
# Criação de uma nova coluna ('Vagas_Ofertadas_calc') que prioriza o valor oficial de # 'Vagas_Ofertadas',
# mas utiliza a soma reconstruída da etapa 1 como alternativa (fallback) caso o valor oficial não seja válido.
# Isto garante que não haja perda de dados.
# Verifica se 'Vagas_Ofertadas' é do tipo Int64 antes do cálculo
if 'Vagas_Ofertadas' in df_tratado.columns:
    df_tratado['Vagas_Ofertadas'] = pd.to_numeric(df_tratado['Vagas_Ofertadas'], errors='coerce').astype('Int64')

# Aplica o método .where() do Pandas para manipulação correta de 'Int64'
# Condição: 'Vagas_Ofertadas' is not null AND > 0
condition = df_tratado["Vagas_Ofertadas"].notna() & (df_tratado["Vagas_Ofertadas"] > 0)
df_tratado["Vagas_Ofertadas_calc"] = df_tratado["Vagas_Ofertadas"].where(
    condition,
    df_tratado["Vagas_categorias_soma"]
).astype("Int64")

# --- PASSO 3: Desduplicação dos Valores por Ciclo de Matrícula ---
# Nos dados brutos, o número de vagas e inscritos de um ciclo se repete para CADA aluno matriculado,
# inflacionando a soma direta. O código abaixo resolve isso atribuindo o valor total de vagas e inscritos
# APENAS à primeira linha de cada ciclo, zerando as demais. Assim, uma simples SOMA dessas novas colunas
# no Looker Studio resultará no valor correto.

# Certifica se 'Total_de_Inscritos' é 'Int64'
if 'Total_de_Inscritos' in df_tratado.columns:
     df_tratado['Total_de_Inscritos'] = pd.to_numeric(df_tratado['Total_de_Inscritos'], errors='coerce').astype('Int64')

# Agrupa o DataFrame por ciclo de matrícula.
g = df_tratado.groupby("Codigo_do_Ciclo_Matricula", dropna=False)

# Para cada ciclo, encontra o valor máximo (e correto) de vagas e inscritos.
# A função 'transform' aplica esse valor a todas as linhas pertencentes ao mesmo ciclo.
vagas_ciclo = g["Vagas_Ofertadas_calc"].transform("max")
inscritos_ciclo = g["Total_de_Inscritos"].transform("max")

# Identifica qual linha é a primeira de cada ciclo de matrícula.
primeira_linha_ciclo = ~df_tratado.duplicated(subset=["Codigo_do_Ciclo_Matricula"], keep="first")

# Cria as colunas finais 'Vagas' e 'Inscritos' com o dtype correto, atribuição usando .loc
df_tratado["Vagas"] = pd.Series(0, index=df_tratado.index, dtype='Int64') # Inicializar com '0' do tipo correto
df_tratado.loc[primeira_linha_ciclo, "Vagas"] = vagas_ciclo[primeira_linha_ciclo] # Atribuir diretamente da série filtrada

df_tratado["Inscritos"] = pd.Series(0, index=df_tratado.index, dtype='Int64') # Inicializar com '0' do tipo correto
df_tratado.loc[primeira_linha_ciclo, "Inscritos"] = inscritos_ciclo[primeira_linha_ciclo] # Atribuir diretamente da série filtrada

# --- PASSO 4: Padronização da Coluna 'Ano' ---
# O campo 'Ano' original da PNP se refere ao ano da extração.
# Este código cria um novo campo 'Ano_de_início_do_ciclo' baseado na data de início do ciclo, o que é
# metodologicamente mais correto para analisar coortes e comparar vagas/inscritos.
df_tratado["Ano_de_início_do_ciclo"] = pd.to_datetime(df_tratado["Data_de_Inicio_do_Ciclo"], errors='coerce').dt.year.astype('Int64')

# Exibição de ´df_tratado´
print(df_tratado[['Codigo_do_Ciclo_Matricula', 'Vagas_Ofertadas_calc', 'Vagas', 'Total_de_Inscritos', 'Inscritos', 'Ano']].head(10))
print(f"Soma correta de Vagas: {df_tratado['Vagas'].sum()}")
print(f"Soma correta de Inscritos: {df_tratado['Inscritos'].sum()}")

In [ ]:
# @title **ETAPA 9: Exibição de estatísticas para colunas selecionadas de `df_tratado`**

if df_tratado is not None and not df_tratado.empty:
    columns_to_describe = [
        'Idade',
        'Total_de_Inscritos',
        'Vagas_Extraordinarias_AC',
        'Vagas_Extraordinarias_l1',
        'Vagas_Extraordinarias_l10',
        'Vagas_Extraordinarias_l13',
        'Vagas_Extraordinarias_l14',
        'Vagas_Extraordinarias_l2',
        'Vagas_Extraordinarias_l5',
        'Vagas_Extraordinarias_l6',
        'Vagas_Extraordinarias_l9',
        'Vagas_Ofertadas',
        'Vagas_Regulares_AC',
        'Vagas_Regulares_l1',
        'Vagas_Regulares_l10',
        'Vagas_Regulares_l13',
        'Vagas_Regulares_l14',
        'Vagas_Regulares_l2',
        'Vagas_Regulares_l5',
        'Vagas_Regulares_l6',
        'Vagas_Regulares_l9',
    ]

    # Filtrar apenas colunas existentes no DataFrame
    existing_columns = [col for col in columns_to_describe if col in df_tratado.columns]

    if existing_columns:
        print("Estatísticas Descritivas para Colunas Selecionadas:")
        # Tente descrever colunas numéricas
        try:
            # Definir opção de formato 'float' para exibição
            pd.options.display.float_format = '{:,.4f}'.format
            display(df_tratado[existing_columns].describe())
        except Exception as e:
            print(f"Erro ao gerar estatísticas para colunas numéricas selecionadas: {e}")

        # Tente descrever colunas de objetos
        try:
            object_columns = df_tratado[existing_columns].select_dtypes(include='object').columns.tolist()
            if object_columns:
                 print("\nEstatísticas Descritivas para Colunas de Objeto Selecionadas:")
                 display(df_tratado[object_columns].describe())
        except Exception as e:
             print(f"Erro ao gerar estatísticas para colunas de objetos selecionadas: {e}")

        # Remove a opção de formato float após a exibição, se desejar, para não afetar outras exibições
        pd.options.display.float_format = None

    else:
        print("Nenhuma das colunas selecionadas foi encontrada no DataFrame.")

elif df_tratado is not None and df_tratado.empty:
    print("O DataFrame 'df_tratado' está vazio, nenhuma estatística para exibir.")
else:
    print("DataFrame 'df_tratado' não encontrado, execute os passos anteriores.")

In [ ]:
# @title **Análise Estatística de _outliers_ em colunas selecionadas de `df_tratado`**

def analyze_outliers_iqr(df: pd.DataFrame, columns: list):
    """
    Calcula e exibe os limites superior e inferior para identificação de outliers
    usando o método IQR para as colunas numéricas especificadas.

    Args:
        df (pd.DataFrame): O DataFrame a ser analisado.
        columns (list): Uma lista de nomes de colunas numéricas para análise.
    """
    if df is None or df.empty:
        print("DataFrame vazio ou não encontrado. Nenhuma análise de outlier realizada.")
        return

    print("--- Análise de Outliers (Método IQR) ---")

    for col in columns:
        if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
            # Remove valores nulos para o cálculo do IQR
            data = df[col].dropna()

            if len(data) < 4: # Precisa de pelo menos 4 pontos para calcular Q1, Q3 e IQR
                print(f"  - Coluna '{col}': Não há dados suficientes para análise de outlier.")
                continue

            Q1 = data.quantile(0.25)
            Q3 = data.quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            print(f"\n--- Coluna: '{col}' ---")
            print(f"  Q1 (25º Percentil): {Q1:,.4f}")
            print(f"  Q3 (75º Percentil): {Q3:,.4f}")
            print(f"  IQR (Intervalo Interquartil): {IQR:,.4f}")
            print(f"  Limite Inferior (Q1 - 1.5*IQR): {lower_bound:,.4f}")
            print(f"  Limite Superior (Q3 + 1.5*IQR): {upper_bound:,.4f}")

            # Opcional: Contar e exibir outliers
            outliers_lower = data[data < lower_bound]
            outliers_upper = data[data > upper_bound]
            total_outliers = len(outliers_lower) + len(outliers_upper)

            print(f"  Número de potenciais outliers abaixo do limite inferior: {len(outliers_lower):,}")
            print(f"  Número de potenciais outliers acima do limite superior: {len(outliers_upper):,}")
            print(f"  Total de potenciais outliers: {total_outliers:,}")

        elif col in df.columns:
             print(f"  - Coluna '{col}': Não é um tipo de dado numérico. Pulando análise de outlier.")
        else:
            print(f"  - AVISO: Coluna '{col}' não encontrada no DataFrame.")

# Lista de colunas para analisar outliers
columns_for_outlier_analysis = [
    'Idade',
    'Total_de_Inscritos',
    'Vagas_Extraordinarias_AC',
    'Vagas_Extraordinarias_l1',
    'Vagas_Extraordinarias_l10',
    'Vagas_Extraordinarias_l13',
    'Vagas_Extraordinarias_l14',
    'Vagas_Extraordinarias_l2',
    'Vagas_Extraordinarias_l5',
    'Vagas_Extraordinarias_l6',
    'Vagas_Extraordinarias_l9',
    'Vagas_Ofertadas',
    'Vagas_Regulares_AC',
    'Vagas_Regulares_l1',
    'Vagas_Regulares_l10',
    'Vagas_Regulares_l13',
    'Vagas_Regulares_l14',
    'Vagas_Regulares_l2',
    'Vagas_Regulares_l5',
    'Vagas_Regulares_l6',
    'Vagas_Regulares_l9',
]

# Executa a análise
analyze_outliers_iqr(df_tratado, columns_for_outlier_analysis)

In [ ]:
# @title **Exibição das informações sobre as colunas e tipos de dados de `df_tratado` finalizado para exportação**
if df_tratado is not None:
    print("Informações sobre as colunas e tipos de dados do DataFrame 'df_tratado':")
    df_tratado.info()
else:
    print("DataFrame 'df_tratado' não encontrado. Execute as etapas anteriores.")

In [ ]:
# @title **ETAPA 10: Exportação para o BigQuery**

# --- Configurações de Destino do BigQuery ---
import pandas_gbq

project_id = "pnp-data-extraction" # Substitua pelo ID do seu projeto
dataset_id = "pnp_dados_IFRN"      # Nome do conjunto de dados

# O nome da tabela no BigQuery será o mesmo nome da tabela da PNP
table_id = f"df_{table_name}"
destination_table = f"{dataset_id}.{table_id}"

# --- Execução da Exportação ---
if df_tratado is not None and not df_tratado.empty:
    print(f"Iniciando a exportação de {len(df_tratado):,} linhas para o BigQuery...")
    print(f"Destino: {project_id}.{destination_table}")

    # --- Renomear colunas para serem compatíveis com BigQuery ---
    # Substitui espaços e '/' por '_'
    df_tratado_bq = df_tratado.copy()
    df_tratado_bq.columns = (df_tratado_bq.columns
            .str.replace(' ', '_', regex=False)
            .str.replace('/', '_', regex=False)
            .str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
        )
    print("Nomes das colunas padronizados para o BigQuery.")

    # --- Definição do Esquema da Tabela ---
    # Defina o nome e o tipo de cada coluna que você quer controlar.
    # Tipos comuns: 'STRING', 'INTEGER', 'FLOAT', 'NUMERIC', 'BOOLEAN', 'TIMESTAMP', 'DATE'.
    table_schema = [
        {"name": "Co_Inst", "type": "STRING"},
        {"name": "Instituicao", "type": "STRING"},
        {"name": "Cod_Unidade", "type": "STRING"},
        {"name": "Campus_do_IFRN", "type": "STRING"},
        {"name": "Codigo_da_Unidade_de_Ensino_-_SISTEC", "type": "STRING"},
        {"name": "Codigo_do_Ciclo_Matricula", "type": "STRING"},
        {"name": "Codigo_da_Matricula", "type": "STRING"},
        {"name": "Municipio", "type": "STRING"},
        {"name": "Codigo_do_Municipio_com_DV", "type": "STRING"},
        {"name": "UF", "type": "STRING"},
        {"name": "Regiao", "type": "STRING"},
        {"name": "Modalidade_Educacional", "type": "STRING"},
        {"name": "Nivel_Educacional", "type": "STRING"},
        {"name": "Modalidade_de_Curso", "type": "STRING"},
        {"name": "Eixo_Tecnologico", "type": "STRING"},
        {"name": "Subeixo_Tecnologico", "type": "STRING"},
        {"name": "Nome_de_Curso", "type": "STRING"},
        {"name": "Cursos", "type": "STRING"},
        {"name": "Turno", "type": "STRING"},
        {"name": "Carga_Horaria", "type": "INTEGER"},
        {"name": "Carga_Horaria_Minima", "type": "INTEGER"},
        {"name": "Fonte_de_Financiamento", "type": "STRING"},
        {"name": "Fator_Esforco_Curso", "type": "FLOAT"},
        {"name": "Sexo", "type": "STRING"},
        {"name": "Cor_Raça", "type": "STRING"},
        {"name": "Idade", "type": "INTEGER"},
        {"name": "Faixa_Etaria", "type": "STRING"},
        {"name": "Renda_Familiar", "type": "STRING"},
        {"name": "Ano", "type": "INTEGER"},
        {"name": "Matricula_Atendida", "type": "STRING"},
        {"name": "Situacao_de_Matricula", "type": "STRING"},
        {"name": "Categoria_da_Situacao", "type": "STRING"},
        {"name": "Data_de_Inicio_do_Ciclo", "type": "DATETIME"},
        {"name": "Data_de_Fim_Previsto_do_Ciclo", "type": "DATETIME"},
        {"name": "Data_de_Ocorrencia_da_Matricula", "type": "DATETIME"},
        {"name": "Mes_De_Ocorrencia_da_Situacao", "type": "STRING"},
        {"name": "Total_de_Inscritos", "type": "INTEGER"},
        {"name": "Forma_de_Ingresso", "type": "STRING"},
        {"name": "Vagas_Ofertadas", "type": "INTEGER"},
        {"name": "Vagas_Regulares_AC", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l1", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l2", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l5", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l6", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l9", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l10", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l13", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l14", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_AC", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l1", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l2", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l5", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l6", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l9", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l10", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l13", "type": "INTEGER"},
        {"name": "Vagas_categorias_soma", "type": "INTEGER"},
        {"name": "Vagas_Ofertadas_calc", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l14", "type": "INTEGER"},
        {"name": "Vagas", "type": "INTEGER"},
        {"name": "Inscritos", "type": "INTEGER"},
        {"name": "Ano_de_início_do_ciclo", "type": "INTEGER"}
    ]
    # -----------------------------------------------------------------

    # Envia o DataFrame para o BigQuery com o esquema especificado
    try:
        # Usando a função recomendada pandas_gbq.to_gbq
        pandas_gbq.to_gbq(
            df_tratado_bq, # Exporta o DataFrame com colunas renomeadas
            destination_table=destination_table,
            project_id=project_id,
            if_exists='replace',  # Opções: 'fail', 'replace', 'append'
            progress_bar=True
        )
        print(f"\n✔ DataFrame exportado com sucesso para o BigQuery!")
        print(f"Link para a tabela: https://console.cloud.google.com/bigquery?project={project_id}&ws=!1m5!1m4!4m3!1s{project_id}!2s{dataset_id}!3s{table_id}")
    except Exception as e:
        print(f"❌ ERRO durante a exportação para o BigQuery: {e}")

elif df_tratado is not None and df_tratado.empty:
    print("AVISO: O DataFrame final está vazio. Nenhuma exportação foi realizada.")
else:
    print("❌ ERRO: O DataFrame a ser exportado não foi encontrado. A exportação foi cancelada.")